# spaCy Text Classification - Reducción de Overfitting

## Objetivo
Probar spaCy TextCategorizer como alternativa a modelos clásicos para reducir overfitting manteniendo F1-score > 0.55.

## Ventajas de spaCy Text Classification
- ✅✅✅ Word embeddings pre-entrenados (mejor semántica que TF-IDF)
- ✅✅✅ Modelo específico para clasificación de texto
- ✅✅✅ Fine-tuning con tu dataset
- ✅✅✅ Mejor control de overfitting que modelos clásicos
- ✅✅✅ Más rápido que Transformers
- ✅ Ya tienes spaCy instalado


## 1. Importación de librerías


In [15]:
import pandas as pd
import numpy as np
import pickle
import random
from pathlib import Path

import spacy
from spacy.util import minibatch, compounding
from spacy.training import Example

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix
)
from sklearn.model_selection import StratifiedKFold

import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
random.seed(42)

print("✅ Librerías importadas")


✅ Librerías importadas


## 2. Cargar modelo de spaCy


In [16]:
# Intentar cargar modelo con word vectors (md o lg)
# Si no está disponible, usar sm y descargar md
try:
    nlp = spacy.load('en_core_web_md')
    print("✅ Modelo en_core_web_md cargado (con word vectors)")
except OSError:
    try:
        nlp = spacy.load('en_core_web_lg')
        print("✅ Modelo en_core_web_lg cargado (con word vectors)")
    except OSError:
        print("⚠️  Modelos con word vectors no encontrados")
        print("   Usando en_core_web_sm (sin word vectors)")
        print("   Para mejor rendimiento, instala: python -m spacy download en_core_web_md")
        nlp = spacy.load('en_core_web_sm')


⚠️  Modelos con word vectors no encontrados
   Usando en_core_web_sm (sin word vectors)
   Para mejor rendimiento, instala: python -m spacy download en_core_web_md


## 3. Carga de datos


In [17]:
# Cargar datos
df = pd.read_csv('../data/processed/youtoxic_english_1000_processed.csv')
with open('../data/processed/y_train.pkl', 'rb') as f:
    y_train = pickle.load(f)
with open('../data/processed/y_test.pkl', 'rb') as f:
    y_test = pickle.load(f)

X_train_text = df[df.index.isin(range(len(y_train)))]['Text_processed'].values
X_test_text = df[df.index.isin(range(len(y_train), len(y_train) + len(y_test)))]['Text_processed'].values

print(f"✅ Datos cargados: {len(X_train_text)} train, {len(X_test_text)} test")
print(f"Distribución train: {np.bincount(y_train)}")
print(f"Distribución test: {np.bincount(y_test)}")


✅ Datos cargados: 800 train, 200 test
Distribución train: [430 370]
Distribución test: [108  92]


## 4. Preparar datos en formato spaCy


In [18]:
# Preparar datos en formato spaCy
# spaCy espera: (texto, {"cats": {"TOXIC": 1.0, "NOT_TOXIC": 0.0}})

def prepare_spacy_data(texts, labels):
    """Prepara datos en formato spaCy para TextCategorizer."""
    data = []
    for text, label in zip(texts, labels):
        # Convertir label a categorías
        cats = {
            "TOXIC": 1.0 if label == 1 else 0.0,
            "NOT_TOXIC": 1.0 if label == 0 else 0.0
        }
        data.append((text, {"cats": cats}))
    return data

# Preparar datos de entrenamiento y prueba
train_data = prepare_spacy_data(X_train_text, y_train)
test_data = prepare_spacy_data(X_test_text, y_test)

print(f"✅ Datos preparados: {len(train_data)} train, {len(test_data)} test")
print(f"Ejemplo de formato:")
print(f"  Texto: {train_data[0][0][:50]}...")
print(f"  Categorías: {train_data[0][1]}")


✅ Datos preparados: 800 train, 200 test
Ejemplo de formato:
  Texto: people would take step back make case be not anyon...
  Categorías: {'cats': {'TOXIC': 1.0, 'NOT_TOXIC': 0.0}}


## 5. Crear y configurar TextCategorizer


In [19]:
# Crear pipeline de spaCy con TextCategorizer
if "textcat" not in nlp.pipe_names:
    # Añadir TextCategorizer al pipeline
    textcat = nlp.add_pipe("textcat", last=True)
else:
    textcat = nlp.get_pipe("textcat")

# Añadir etiquetas
textcat.add_label("TOXIC")
textcat.add_label("NOT_TOXIC")

print("✅ TextCategorizer configurado")
print(f"Etiquetas: {textcat.labels}")


✅ TextCategorizer configurado
Etiquetas: ('TOXIC', 'NOT_TOXIC')


## 6. Función de evaluación


In [20]:
def evaluate_spacy_model(nlp_model, texts, true_labels):
    """Evalúa modelo de spaCy y retorna métricas."""
    predictions = []
    
    # Procesar textos y obtener predicciones
    for text in texts:
        doc = nlp_model(text)
        # Obtener probabilidad de clase TOXIC
        toxic_score = doc.cats.get("TOXIC", 0.0)
        # Predicción: 1 si TOXIC > 0.5, 0 en caso contrario
        pred = 1 if toxic_score > 0.5 else 0
        predictions.append(pred)
    
    predictions = np.array(predictions)
    true_labels = np.array(true_labels)
    
    # Calcular métricas
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, zero_division=0)
    recall = recall_score(true_labels, predictions, zero_division=0)
    f1 = f1_score(true_labels, predictions, zero_division=0)
    cm = confusion_matrix(true_labels, predictions)
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'confusion_matrix': cm,
        'predictions': predictions
    }

print("✅ Función de evaluación definida")


✅ Función de evaluación definida


## 7. Entrenar modelo


In [ ]:
# Convertir datos a formato Example de spaCy
train_examples = [Example.from_dict(nlp.make_doc(text), annots) for text, annots in train_data]

# Deshabilitar otros componentes del pipeline durante entrenamiento
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "textcat"]

print("="*80)
print("ENTRENAMIENTO SPACY TEXTCATEGORIZER")
print("="*80)
print(f"Componentes deshabilitados: {other_pipes}")
print(f"Épocas: 10")
print(f"Batch size: compuesto (4-32)")
print("-"*80)

# Entrenar modelo
n_iter = 10
with nlp.disable_pipes(*other_pipes):
    # Inicializar el componente
    # Si hay problemas con lookups, intentar sin ellos
    try:
        nlp.initialize(lambda: train_examples)
    except ValueError as e:
        if "lookups" in str(e).lower() or "lookup" in str(e).lower():
            print("⚠️  Advertencia: Problemas con lookups, inicializando sin ellos...")
            try:
                nlp.initialize(lambda: train_examples, exclude=["lookups"])
            except:
                # Si aún falla, usar método alternativo
                print("⚠️  Usando método alternativo de inicialización...")
                nlp.initialize(get_examples=lambda: train_examples)
        else:
            raise
    
    # Entrenar
    for epoch in range(n_iter):
        # Mezclar datos
        random.shuffle(train_examples)
        
        # Crear batches de tamaño variable
        batches = minibatch(train_examples, size=compounding(4.0, 32.0, 1.001))
        
        losses = {}
        for batch in batches:
            nlp.update(batch, losses=losses, drop=0.2)  # drop=0.2 para regularización
        
        if epoch % 2 == 0:
            print(f"Época {epoch+1}/{n_iter} - Loss: {losses.get('textcat', 0.0):.4f}")

print("\n✅ Entrenamiento completado")


ENTRENAMIENTO SPACY TEXTCATEGORIZER
Componentes deshabilitados: ['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']
Épocas: 10
Batch size: compuesto (4-32)
--------------------------------------------------------------------------------
⚠️  Advertencia: Problemas con lookups, inicializando sin ellos...
⚠️  Usando método alternativo de inicialización...


ValueError: [E955] Can't find table(s) lexeme_norm for language 'en' in spacy-lookups-data. Make sure you have the package installed or provide your own lookup tables if no default lookups are available for your language.

## 8. Evaluación del modelo


In [ ]:
# Evaluar en train y test
print("Evaluando en conjunto de entrenamiento...")
train_results = evaluate_spacy_model(nlp, X_train_text, y_train)

print("\nEvaluando en conjunto de prueba...")
test_results = evaluate_spacy_model(nlp, X_test_text, y_test)

# Calcular diferencia de F1 (overfitting)
diff_f1 = abs(train_results['f1'] - test_results['f1']) * 100

print("\n" + "="*80)
print("RESULTADOS FINALES - SPACY TEXTCATEGORIZER")
print("="*80)
print(f"F1-score (train): {train_results['f1']:.4f}")
print(f"F1-score (test): {test_results['f1']:.4f}")
print(f"Accuracy (test): {test_results['accuracy']:.4f}")
print(f"Precision (test): {test_results['precision']:.4f}")
print(f"Recall (test): {test_results['recall']:.4f}")
print(f"Diferencia F1: {diff_f1:.2f}%")
print(f"\nMatriz de confusión (test):")
print(test_results['confusion_matrix'])

if diff_f1 < 5.0 and test_results['f1'] > 0.55:
    print("\n✅✅✅ OBJETIVO CUMPLIDO: Overfitting < 5% Y F1 > 0.55")
elif diff_f1 < 6.0:
    print("\n🎯 MUY CERCA: Overfitting < 6%")
else:
    print("\n⚠️  Overfitting aún alto")

print("="*80)


## 9. Validación Cruzada (opcional)


In [ ]:
# Validación cruzada (toma tiempo, opcional)
print("Validación cruzada (5-fold)...")
print("⚠️  Esto puede tardar varios minutos...")

X_all_text = np.concatenate([X_train_text, X_test_text])
y_all = np.concatenate([y_train, y_test])

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = []

for fold, (train_idx, val_idx) in enumerate(cv.split(X_all_text, y_all), 1):
    print(f"Fold {fold}/5...")
    
    # Preparar datos del fold
    fold_train_data = prepare_spacy_data(X_all_text[train_idx], y_all[train_idx])
    fold_val_data = prepare_spacy_data(X_all_text[val_idx], y_all[val_idx])
    
    # Crear nuevo modelo para este fold
    fold_nlp = spacy.load('en_core_web_md' if 'md' in str(nlp.meta.get('name', '')) else 'en_core_web_sm')
    if "textcat" not in fold_nlp.pipe_names:
        fold_textcat = fold_nlp.add_pipe("textcat", last=True)
    else:
        fold_textcat = fold_nlp.get_pipe("textcat")
    fold_textcat.add_label("TOXIC")
    fold_textcat.add_label("NOT_TOXIC")
    
    # Entrenar
    fold_examples = [Example.from_dict(fold_nlp.make_doc(text), annots) for text, annots in fold_train_data]
    fold_other_pipes = [pipe for pipe in fold_nlp.pipe_names if pipe != "textcat"]
    
    with fold_nlp.disable_pipes(*fold_other_pipes):
        fold_nlp.initialize(lambda: fold_examples)
        for epoch in range(5):  # Menos épocas para CV
            random.shuffle(fold_examples)
            batches = minibatch(fold_examples, size=compounding(4.0, 32.0, 1.001))
            for batch in batches:
                fold_nlp.update(batch, losses={}, drop=0.2)
    
    # Evaluar
    fold_results = evaluate_spacy_model(fold_nlp, X_all_text[val_idx], y_all[val_idx])
    cv_scores.append(fold_results['f1'])
    print(f"  F1-score: {fold_results['f1']:.4f}")

print(f"\nF1-score (CV): {np.mean(cv_scores):.4f} (+/- {np.std(cv_scores) * 2:.4f})")
print(f"Scores: {cv_scores}")


## 10. Guardar modelo (si cumple objetivos)


In [ ]:
if diff_f1 < 6.0 and test_results['f1'] > 0.55:
    # Guardar modelo
    output_dir = Path('../models/spacy_textcat_model')
    output_dir.mkdir(parents=True, exist_ok=True)
    nlp.to_disk(output_dir)
    
    # Guardar información del modelo
    model_info = {
        'model_type': 'spaCy TextCategorizer',
        'spacy_model': str(nlp.meta.get('name', 'unknown')),
        'test_f1': test_results['f1'],
        'diff_f1': diff_f1,
        'n_iter': n_iter,
        'has_word_vectors': hasattr(nlp.vocab.vectors, 'shape')
    }
    
    with open('../models/spacy_textcat_info.pkl', 'wb') as f:
        pickle.dump(model_info, f)
    
    print(f"✅ Modelo guardado en: {output_dir}")
    print(f"✅ Información guardada en: ../models/spacy_textcat_info.pkl")
else:
    print("⚠️  Modelo no guardado (no cumple objetivos)")
